In [39]:
#import statements

%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set() 
from sklearn.cluster import KMeans 
from sklearn.neighbors import KNeighborsClassifier
from scipy.spatial import distance
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import random as sparse_random
from numpy import random

In [46]:
#import data


movies_pd = pd.read_csv("merged_movie_data.csv", sep = ",")
movies_np = movies_pd.to_numpy()

ratings_pd = pd.read_csv("wrangled_ratings_data.csv", sep = ",")
ratings_np = ratings_pd.to_numpy()


C:\Users\Elise\anaconda3\envs\csc294\lib\site-packages\IPython\core\interactiveshell.py:3155: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [36]:
#system without any adjustments

def movie_rec_system(df, n_components, recs_num):

    
    #this inputed data has already been well formated for this type of recommender system
    #formatting of data has happened in other document


    #already changed all null values (0) to a neutral value on a scale of 5 (2.5) already in the data wrangling process

    #pushing data into lower dimension 

    svd = TruncatedSVD(n_components = n_components, n_iter=7, random_state=42)
    df_fit = svd.fit(df)
    transformtest = svd.fit_transform(df)
    
    #pushing the data back to its orginal size from the lower dimension data

    final_ratings = svd.inverse_transform(transformtest)

    original_size = final_ratings

    output_recs = []

    #sort through ratings to see which are very positive and have not already been rated

    #all already rated movies are 0
    for x in range(len(original_size[:,1])):
        for y in range(len(original_size[1,:])):
            if (df[x,y]!=2.5):
                original_size[x,y] = 0


    #get recs_num max ratings
    
    for x in range(len(original_size[:,1])):
     
        recs = []

        for c in range(recs_num):

            max_num = max(original_size[x,:])

            for y in range(len(original_size[1,:])):
                movieid = y+1
                
                if (original_size[x,y] == max_num) and df[x,y]==2.5:
                    new_rec = []
                    original_size[x,y] = 1  
                    recs.append(movieid)
                
        output_recs.append(recs) 

    #original favs are IMDB movie ids of movies that were rated 4 or 5 by the user
    original_favs = []
    
    for user in range(len(df[:,0])):
        user_og_favs = []
        for movie in range(len(df[:,0])):
            if df[user, movie] >= 4:
                user_og_favs.append(movie+1)
    
        original_favs.append(user_og_favs)
    
    
    #both original_favs and output_recs should be length of the number of users

    finaloutput = [original_favs, output_recs]

    return finaloutput

In [35]:
#system with adjustments for audience count

def movie_rec_system_audience(df, info_df, n_components, recs_num, adjust):

    
    #this inputed data has already been well formated for this type of recommender system
    #formatting of data has happened in other document


    #already changed all null values (0) to a neutral value on a scale of 5 (2.5) already in the data wrangling process

    #pushing data into lower dimension

    svd = TruncatedSVD(n_components = n_components, n_iter=7, random_state=42)
    df_fit = svd.fit(df)
    transformtest = svd.fit_transform(df)
    
    #pushing the data back to its orginal size from the lower dimension data

    final_ratings = svd.inverse_transform(transformtest)

    original_size = final_ratings

    output_recs = []

    #sort through ratings to see which are very positive and have not already been rated

    #all already rated books are 0
    for x in range(len(original_size[:,1])):
        for y in range(len(original_size[1,:])):
            if (df[x,y]!=2.5):
                original_size[x,y] = 0

                
    #adjusting for audience count so highly watched movies do not dominate the recommendation
                
    for ind in range(len(info_df[:,0])):
        if info_df[ind,0] < len(original_size[0,:]):
            if info_df[ind, 1] > 20000000:
                original_size[:,ind] = original_size[:,ind] - adjust
            else:
                original_size[:,ind] = original_size[:,ind] + adjust
                
          

    #get recs_num max ratings
    
    for x in range(len(original_size[:,1])):
     
        recs = []

        for c in range(recs_num):

            max_num = max(original_size[x,:])

            for y in range(len(original_size[1,:])):
                movieid = y+1
                
                if (original_size[x,y] == max_num) and df[x,y]==2.5:
                    new_rec = []
                    original_size[x,y] = 1  
                    recs.append(movieid)
                
        output_recs.append(recs) 

    #original favs are IMDB movie ids of movies that were rated 4 or 5 by the user
    original_favs = []
    
    for user in range(len(df[:,0])):
        user_og_favs = []
        for movie in range(len(df[:,0])):
            if df[user, movie] >= 4:
                user_og_favs.append(movie+1)
    
        original_favs.append(user_og_favs)
    
    
    #both original_favs and output_recs should be length of the number of users

    finaloutput = [original_favs, output_recs]

    return finaloutput

In [37]:
#system with adjustments for bechdel test

def movie_rec_system_bechdel(df, info_df, n_components, recs_num, adjust):

    
    #this inputed data has already been well formated for this type of recommender system
    #formatting of data has happened in other document


    #already changed all null values (0) to a neutral value on a scale of 5 (2.5) already in the data wrangling process

    #pushing data into lower dimension

    svd = TruncatedSVD(n_components = n_components, n_iter=7, random_state=42)
    df_fit = svd.fit(df)
    transformtest = svd.fit_transform(df)
    
    #pushing the data back to its orginal size from the lower dimension data

    final_ratings = svd.inverse_transform(transformtest)

    original_size = final_ratings

    output_recs = []

    #sort through ratings to see which are very positive and have not already been rated

    #all already rated books are 0
    for x in range(len(original_size[:,1])):
        for y in range(len(original_size[1,:])):
            if (df[x,y]!=2.5):
                original_size[x,y] = 0

                
    #adjusting for audience count for bechdel test
                
    for ind in range(len(info_df[:,0])):
        if info_df[ind,0] < len(original_size[0,:]):
            if info_df[ind, 2] == 0:
                original_size[:,ind] = original_size[:,ind] - adjust
            else:
                original_size[:,ind] = original_size[:,ind] + adjust
                
          

    #get recs_num max ratings
    
    for x in range(len(original_size[:,1])):
     
        recs = []

        for c in range(recs_num):

            max_num = max(original_size[x,:])

            for y in range(len(original_size[1,:])):
                movieid = y+1
                
                if (original_size[x,y] == max_num) and df[x,y]==2.5:
                    new_rec = []
                    original_size[x,y] = 1  
                    recs.append(movieid)
                
        output_recs.append(recs) 

    #original favs are IMDB movie ids of movies that were rated 4 or 5 by the user
    original_favs = []
    
    for user in range(len(df[:,0])):
        user_og_favs = []
        for movie in range(len(df[:,0])):
            if df[user, movie] >= 4:
                user_og_favs.append(movie+1)
    
        original_favs.append(user_og_favs)
    
    
    #both original_favs and output_recs should be length of the number of users

    finaloutput = [original_favs, output_recs]

    return finaloutput

In [38]:
#system with adjustments for bechdel test and audience count

def movie_rec_system_audience_bechdel(df, info_df, n_components, recs_num, adjust_a, adjust_b):

    
    #this inputed data has already been well formated for this type of recommender system
    #formatting of data has happened in other document


    #already changed all null values (0) to a neutral value on a scale of 5 (2.5) already in the data wrangling process

    #pushing data into lower dimension

    svd = TruncatedSVD(n_components = n_components, n_iter=7, random_state=42)
    df_fit = svd.fit(df)
    transformtest = svd.fit_transform(df)
    
    #pushing the data back to its orginal size from the lower dimension data

    final_ratings = svd.inverse_transform(transformtest)

    original_size = final_ratings

    output_recs = []

    #sort through ratings to see which are very positive and have not already been rated

    #all already rated books are 0
    for x in range(len(original_size[:,1])):
        for y in range(len(original_size[1,:])):
            if (df[x,y]!=2.5):
                original_size[x,y] = 0

                
    #adjusting for audience count for bechdel test and audience count
                
    for ind in range(len(info_df[:,0])):
        if info_df[ind,0] < len(original_size[0,:]):
            if info_df[ind, 1] > 20000000:
                original_size[:,ind] = original_size[:,ind] - adjust_a
            else:
                original_size[:,ind] = original_size[:,ind] + adjust_a
            if info_df[ind, 2] == 0:
                original_size[:,ind] = original_size[:,ind] - adjust_b
            else:
                original_size[:,ind] = original_size[:,ind] + adjust_b
                
          

    #get recs_num max ratings
    
    for x in range(len(original_size[:,1])):
     
        recs = []

        for c in range(recs_num):

            max_num = max(original_size[x,:])

            for y in range(len(original_size[1,:])):
                movieid = y+1
                
                if (original_size[x,y] == max_num) and df[x,y]==2.5:
                    new_rec = []
                    original_size[x,y] = 1  
                    recs.append(movieid)
                
        output_recs.append(recs) 

    #original favs are IMDB movie ids of movies that were rated 4 or 5 by the user
    original_favs = []
    
    for user in range(len(df[:,0])):
        user_og_favs = []
        for movie in range(len(df[:,0])):
            if df[user, movie] >= 4:
                user_og_favs.append(movie+1)
    
        original_favs.append(user_og_favs)
    
    
    #both original_favs and output_recs should be length of the number of users

    finaloutput = [original_favs, output_recs]

    return finaloutput

In [18]:
print(movies_np.shape)


sorted_movie_np = movies_np[np.argsort(movies_np[:, 0])]


(1730, 3)


In [40]:
smaller_ratings_data = ratings_np[:600,:20000]
normal_movie_recs = movie_rec_system(smaller_ratings_data, 30, 10)

In [ ]:
audience_movie_recs = movie_rec_system_audience(smaller_ratings_data, sorted_movie_np, 30, 10, 0.2)

In [42]:
bechdel_movie_recs = movie_rec_system_bechdel(smaller_ratings_data, sorted_movie_np, 30, 10, 0.2)

In [43]:
movie_recs_both = movie_rec_system_audience_bechdel(smaller_ratings_data, sorted_movie_np, 30, 10, 0.2, 0.2)

In [70]:
rand1 = random.randint(600)
print(rand1)
rand2 = random.randint(600)
print(rand2)
rand3 = random.randint(600)
print(rand3)

417
537
9


In [71]:
#random user 1
likes = normal_movie_recs[0][rand1]
normal = normal_movie_recs[1][rand1]
aud = audience_movie_recs[1][rand1]
bech = bechdel_movie_recs[1][rand1]
both = movie_recs_both[1][rand1]


print(likes)
print(normal)
print(aud)
print(bech)
print(both)



[151, 159, 340, 357, 365, 381, 481, 587, 598]
[4887, 5419, 111, 5350, 5990, 596, 589, 3794, 458, 8666]
[4887, 111, 5419, 596, 589, 5350, 458, 5990, 588, 540]
[4887, 5419, 596, 5350, 5990, 588, 3794, 111, 166, 833]
[4887, 596, 5419, 588, 111, 5350, 833, 5990, 589, 3794]


In [72]:
#random user 2
likes = normal_movie_recs[0][rand2]
normal = normal_movie_recs[1][rand2]
aud = audience_movie_recs[1][rand2]
bech = bechdel_movie_recs[1][rand2]
both = movie_recs_both[1][rand2]

print(likes)
print(normal)
print(aud)
print(bech)
print(both)

[2, 111, 112, 151, 247, 301, 319, 320, 510, 589, 591]
[528, 357, 51, 594, 458, 609, 365, 35, 481, 590]
[528, 357, 51, 594, 458, 609, 365, 35, 481, 590]
[365, 528, 481, 48, 37, 18, 498, 1149, 162, 338]
[365, 528, 481, 48, 18, 498, 1149, 162, 338, 378]


In [73]:
#random user 3
likes = normal_movie_recs[0][rand3]
normal = normal_movie_recs[1][rand3]
aud = audience_movie_recs[1][rand3]
bech = bechdel_movie_recs[1][rand3]
both = movie_recs_both[1][rand3]

print(likes)
print(normal)
print(aud)
print(bech)
print(both)

[51, 153, 319, 346]
[261, 1271, 590, 528, 2029, 297, 542, 357, 1215, 2859]
[261, 590, 528, 542, 357, 609, 1271, 594, 2029, 1137]
[261, 1271, 2029, 1137, 1098, 247, 859, 1149, 37, 1074]
[261, 1137, 1098, 247, 859, 1149, 1074, 672, 224, 1173]
